In [6]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import sys
sys.path.append('../wiki_kb_inference')

import pandas as pd

from fgc_wiki_qa.utils.stanfordnlp_utils import *
from fgc_wiki_qa.utils.fgc_utils import *
from fgc_wiki_qa.utils.utils import load_json
from stanfordnlp.server import CoreNLPClient


In [7]:
from matplotlib.font_manager import FontProperties

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial'] 
plt.rcParams['axes.unicode_minus'] = False




## Load data

In [8]:
train_docs = load_json('../data/external/whole_system/4.2-1/question_train_result.json')
train_ids = [q['QID'] for q, d in q_doc_generator(train_docs)]
dev_docs = load_json('../data/external/whole_system/4.2-1/question_dev_result.json')
dev_ids = [q['QID'] for q, d in q_doc_generator(dev_docs)]
test_docs = load_json('../data/external/whole_system/4.2-1/question_test_result.json')
test_ids = [q['QID'] for q, d in q_doc_generator(test_docs)]
docs = train_docs + dev_docs + test_docs
docs.sort(key=lambda d: d['DID'])

## Aggregator Score Top 1 Result

In [9]:
import re
pred = '他和他、asdf'
re.split(r'及|以及|和|與|、|,|，',pred)

['他', '他', 'asdf']

In [29]:
import re

def pred_in_ans(pred, golds):
    pred_set = set(re.split(r'及|以及|和|與|、|,|，',pred))
    gold_sets = [set(re.split(r'及|以及|和|與|、|,|，', gold)) for gold in golds]
    if pred_set in gold_sets:
        return True
    else:
        return False

def correct_gen(docs):
    for q, doc in q_doc_generator(docs):
        golds = [a['ATEXT']for a in q['ANSWER']]
        pred = q['AFINAL']['ATEXT_TW']
        if pred_in_ans(pred, golds):
            yield q, doc

def error_gen(docs):
    for q, doc in q_doc_generator(docs):
        golds = [a['ATEXT']for a in q['ANSWER']]
        pred = q['AFINAL']['ATEXT_TW']
        if not pred_in_ans(pred, golds):
            yield q, doc

In [27]:
qids = [q['QID'] for q, _ in q_doc_generator(docs)]
amodes = ['Single-Span-Extraction', 'Date-Duration', 'Multi-Spans-Extraction', 'Counting', 'Kinship', 'YesNo', 'Arithmetic-Operations', 'Comparing-Members', 'CommonSense']

In [35]:
ids = [q['QID'] for q, doc in q_doc_generator(docs)]
modules = [q['AFINAL']['AMODULE'] for q, doc in q_doc_generator(docs)]
atypes = [list(q['ATYPE'].keys())[0] for q, doc in q_doc_generator(docs)]
scores = [q['AFINAL']['score'] for q, doc in q_doc_generator(docs)]
preds = [q['AFINAL']['ATEXT_TW'] for q, doc in q_doc_generator(docs)]
golds = ['/'.join(get_golds_from_qid(q['QID'], docs)) for q, doc in q_doc_generator(docs)]
results = [pred_in_ans(q['AFINAL']['ATEXT_TW'], [a['ATEXT']for a in q['ANSWER']]) for q, doc in q_doc_generator(docs)]
df = pd.DataFrame(dict(zip(['qid', 'pred_amodule', 'pred_atype', 'amodule_score', 'pred', 'gold', 'result'], [ids, modules, atypes, scores, preds, golds, results])))

In [39]:
df.to_excel('whole_system_aggr_top1.xlsx', index=None)

## Visual

In [23]:
amod_count_df = df.pivot_table(index=['amodule'], values=['qid'], aggfunc='count').T

error_ids_dict = df.pivot_table(index=['amodule'], values=['qid'], aggfunc=lambda l: l.to_list()).to_dict()['qid']

In [28]:
@interact
def basic_errors(amode=amodes):
    print('=== Errors ===')
    try:
        print('error ids of {}: {l}'.format(amode, l=error_ids_dict[amode]))
    except KeyError:
        print('No errors')
    

interactive(children=(Dropdown(description='amode', options=('Single-Span-Extraction', 'Date-Duration', 'Multi…

In [40]:
@interact
def all_candidates(qid=qids):
    did = qid[:4]
    doc_dic = get_doc(did, docs)
    q_dict = get_que(qid, docs)
    
    
    if qid in train_ids:
        which_split = 'TRAIN'
    elif qid in dev_ids:
        which_split = 'DEV'
    elif qid in test_ids:
        which_split = 'TEST'
    else:
        raise ValueError
        
    print('FROM {} SET'.format(which_split))
    
    snp_print_legends()

    with CoreNLPClient(endpoint='http://140.109.19.51:9000', start_server=False) as nlp:
        
        print('=== Passage ===')
        nlp_props = {'ssplit.boundaryTokenRegex': '[。]|[!?！？]+',
#                'pipelineLanguage': 'zh',
               'annotators': 'tokenize,ssplit,pos,lemma,ner'}
        doc = nlp.annotate(doc_dic['DTEXT_CN'], properties=nlp_props)
        print_props = {
#             'mode': 'custom',
            'classes_w_color': ['PERSON', 'GPE', 'LOCATION', 'MISC', 'TITLE']
        }
        for ix, sent in enumerate(doc.sentence):
            print(f'(s{sent.sentenceIndex})', end=' ')
            print(snp_pstr(sent, **print_props))
#             snp_pprint_by_displacy(sent, doc)
    
        print('\n=== Question ===')
        q = nlp.annotate(q_dict['QTEXT_CN'], properties=nlp_props)
        for sent in q.sentence:
            print(f'(s{sent.sentenceIndex})', end=' ')
            print(snp_pstr(sent, **print_props))
#             snp_pprint_by_displacy(sent, q)
    
    pred_dict = q_dict['AFINAL']
    print('(Pred) {} (by {} - score {})'.format(pred_dict['ATEXT'], pred_dict['AMODULE'], str(pred_dict['score'])))
    print('(Gold)', [a['ATEXT'] for a in q_dict['ANSWER']])
    print()
    amode_dict = [(mode, v['score']) for mode, v in q_dict['AMODE'].items()]
    df = pd.DataFrame(amode_dict, columns=['amode', 'score'])
    ax = df.plot.barh(x='amode', y='score', legend=False, title='AMODE', sort_columns=True)
    for p in ax.patches:
        ax.annotate(f'{p.get_width():.6f}', (p.get_width() * 1.005, p.get_y() * 1.005))
    print(ax)
    
    for amode in amodes:
        amode_ans_dict = get_amode(qid, amode, docs)
        acands = amode_ans_dict['ACAND']
        if len(acands) == 0 or set([c['score'] for c in acands]) == {0}:
            continue
        df = pd.DataFrame(acands)
#         print(df)
        try:
            ax = df.plot.barh(x='ATEXT', y=['score', 'AMODULE'], title=amode, sort_columns=True)
            for p in ax.patches:
                ax.annotate(f'{p.get_width():.6f}', (p.get_width() * 1.005, p.get_y() * 1.005))
        except KeyError:
            raise
    return None

interactive(children=(Dropdown(description='qid', options=('D001Q01', 'D001Q02', 'D001Q03', 'D001Q04', 'D001Q0…